# Consolidação de Arquivos - Múltipos Layouts e Extensões (OBT)

### Criação do modelo com todos os atributos possíveis

In [1]:
import duckdb
import pandas as pd

In [2]:
df_base_atributos = pd.read_csv(
    "../resultados/detalhes/consolidacao_detalhes.csv",
    sep=";",
    dtype="str"
)

In [3]:
df_base_atributos.head(10)

,caminho,arquivo,coluna,amostra,quantidade,aba,observacoes,status,nova_coluna
0,../arquivos,informacao-cargo.txt,Cargo,Gerente;Analista,3,NaN,NaN,OK,***Esse atributo pode renomear as colunas na c...
1,../arquivos,informacao-cargo.txt,ID,1;2,3,NaN,NaN,OK,***Esse atributo pode renomear as colunas na c...
2,../arquivos,informacao-cargo.txt,Idade,30;25,3,NaN,NaN,OK,***Esse atributo pode renomear as colunas na c...
3,../arquivos,informacao-cargo.txt,Nome,Ana;Bruno,3,NaN,NaN,OK,***Esse atributo pode renomear as colunas na c...
4,../arquivos,informacao-empresas.csv,Empresa,Empresa A;Empresa B,3,NaN,NaN,OK,***Esse atributo pode renomear as colunas na c...
5,../arquivos,informacao-empresas.csv,ID,1;2,3,NaN,NaN,OK,***Esse atributo pode renomear as colunas na c...
6,../arquivos,informacao-empresas.csv,Idade,30;25,3,NaN,NaN,OK,***Esse atributo pode renomear as colunas na c...
7,../arquivos,informacao-empresas.csv,Nome,Ana;Bruno,3,NaN,NaN,OK,***Esse atributo pode renomear as colunas na c...
8,../arquivos,informacao-erro.jpg,NaN,NaN,NaN,NaN,Erro: Formato de arquivo não suportado,ERRO,NaN
9,../arquivos,informacao-projeto.xlsx,ID,1;2,3,Sheet1,NaN,OK,***Esse atributo pode renomear as colunas na c...


In [4]:
query = """
SELECT
    coluna,
    amostra
FROM (
    SELECT 
            ROW_NUMBER() OVER (PARTITION BY coluna ORDER BY coluna, amostra) AS ID,
            coluna,
            split_part(amostra, ';', 1) amostra
    FROM df_base_atributos
    WHERE coluna IS NOT NULL
    ORDER BY coluna
    ) T1
WHERE ID = 1;
"""

In [5]:
with duckdb.connect(database=":memory:", read_only=False) as conexao:
    df_modelo_colunas = conexao.sql(query).df()

In [6]:
df_modelo_colunas

,coluna,amostra
0,CPF,111.111.111-11
1,Cargo,Gerente
2,Departamento,RH
3,Email,ana@example.com
4,Empresa,Empresa A
5,Endereco,Rua A 123
6,Estado,SP
7,ID,1
8,Idade,30
9,Nome,Ana


In [7]:
df_modelo_colunas = df_modelo_colunas.pivot(columns="coluna", values="amostra")

In [8]:
#df_modelo_colunas
#df_modelo_colunas.ffill()
df_modelo_colunas.ffill().bfill().iloc[:1]

coluna,CPF,Cargo,Departamento,Email,Empresa,Endereco,Estado,ID,Idade,Nome,Projeto,Salario,Status,Telefone
0,111.111.111-11,Gerente,RH,ana@example.com,Empresa A,Rua A 123,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789


In [9]:
df_modelo_colunas = df_modelo_colunas.ffill().bfill().iloc[:1]

In [10]:
(
    df_modelo_colunas.to_json(
        "../modelo/base/layout_modelo.json",
        orient="records",
        force_ascii=False
    )
)

### Criando o modelo de dados geral

+ Criando o JsonSchema do modelo

In [32]:
import json


schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "TodasColunas",
    "type": "object",
    "properties": {}
}

with open("../modelo/base/layout_modelo.json") as arquivo:
    atributos = json.load(arquivo)
    
for chave in atributos[0].keys():
    schema["properties"][chave] = {"type": "string"}

json_schema = json.dumps(schema, indent=2)

with open("../modelo/base/jsonshema_modelo.json", "w") as arquivo:
    json.dump(schema, arquivo, indent=2)

JSON Schema gerado com sucesso!


+ Criando a Classe Python do modelo

In [41]:
! mkdir -p ./modelo && \
    datamodel-codegen  \
    --input ../modelo/base/jsonshema_modelo.json \
    --input-file-type jsonschema \
    --output ./modelo/modelo.py

### Conversão de todos os arquivos em JSONs válidos.

In [70]:
import os
from pathlib import Path
from secrets import token_hex
from datetime import datetime

import pandas as pd
import duckdb


def gera_arquivo_json(arquivo: str, aba: str, encoding: str ="latin1") -> bool:
    try:
        if arquivo.endswith(".xlsx") or arquivo.endswith(".xls"):
            df = pd.read_excel(arquivo, engine="openpyxl", sheet_name=aba, dtype="str")
        elif arquivo.endswith(".csv") or arquivo.endswith(".txt"):
            df = pd.read_csv(arquivo, encoding=encoding, delimiter=";", dtype="str")
        elif arquivo.endswith(".json"):
            df = pd.read_json(arquivo)
        else:
            raise ValueError("Formato de arquivo não suportado")

        df["caminho_arquivo"] = str(Path(os.path.relpath(arquivo)).parent)
        df["nome_arquivo"] = arquivo.split("/")[-1]
        df["aba_arquivo"] = aba
        df["datahora_processamento"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        df = df.astype("str")

        caminho_arquivo_json = ("../resultados/tmp/jsons")

        if not os.path.exists(caminho_arquivo_json):
            Path(caminho_arquivo_json).mkdir(parents=True, exist_ok=True)

        arquivo_json = f"{arquivo.split('/')[-1].split('.')[0]}-{token_hex(16)}.json"
        df.to_json(f"{caminho_arquivo_json}/{arquivo_json}", orient="records", force_ascii=False)  
        return True

    except Exception as erro:
        raise ValueError(f"Erro na formatação final do arquivo {arquivo}: {erro}")


In [71]:
detalhes_arquivos = pd.read_csv(
    "../resultados/detalhes/consolidacao_detalhes.csv",
    sep=";",
    dtype="str"
)

In [72]:
detalhes_arquivos = detalhes_arquivos.fillna("")

In [73]:
detalhes_arquivos.head(10)

,caminho,arquivo,coluna,amostra,quantidade,aba,observacoes,status,nova_coluna
0,../arquivos,informacao-cargo.txt,Cargo,Gerente;Analista,3,,,OK,***Esse atributo pode renomear as colunas na c...
1,../arquivos,informacao-cargo.txt,ID,1;2,3,,,OK,***Esse atributo pode renomear as colunas na c...
2,../arquivos,informacao-cargo.txt,Idade,30;25,3,,,OK,***Esse atributo pode renomear as colunas na c...
3,../arquivos,informacao-cargo.txt,Nome,Ana;Bruno,3,,,OK,***Esse atributo pode renomear as colunas na c...
4,../arquivos,informacao-empresas.csv,Empresa,Empresa A;Empresa B,3,,,OK,***Esse atributo pode renomear as colunas na c...
5,../arquivos,informacao-empresas.csv,ID,1;2,3,,,OK,***Esse atributo pode renomear as colunas na c...
6,../arquivos,informacao-empresas.csv,Idade,30;25,3,,,OK,***Esse atributo pode renomear as colunas na c...
7,../arquivos,informacao-empresas.csv,Nome,Ana;Bruno,3,,,OK,***Esse atributo pode renomear as colunas na c...
8,../arquivos,informacao-erro.jpg,,,,,Erro: Formato de arquivo não suportado,ERRO,
9,../arquivos,informacao-projeto.xlsx,ID,1;2,3,Sheet1,,OK,***Esse atributo pode renomear as colunas na c...


In [74]:
query = """
SELECT DISTINCT caminho, arquivo, aba
FROM detalhes_arquivos
WHERE status = 'OK';
"""

In [75]:
with duckdb.connect(database=":memory:", read_only=False) as conexao:
    arquivos_processamento = conexao.sql(query).fetchall()

In [76]:
arquivos_processamento

[('../arquivos', 'informacao-projeto.xlsx', 'Sheet2'),
 ('../arquivos/dir1', 'informacao-pessoa.json', ''),
 ('../arquivos/dir2', 'informacoes-contatos.txt', ''),
 ('../arquivos', 'informacao-cargo.txt', ''),
 ('../arquivos', 'informacao-empresas.csv', ''),
 ('../arquivos', 'informacao-projeto.xlsx', 'Sheet1'),
 ('../arquivos/dir2/dir2.1', 'informacao-salario.csv', ''),
 ('../arquivos/dir2', 'informacao-endereco.json', ''),
 ('../arquivos/dir1', 'informacao-departamento.xlsx', 'Sheet1')]

In [77]:
for item in arquivos_processamento:
    arquivo = f"{item[0]}/{item[1]}"
    aba = item[2]
    gera_arquivo_json(arquivo, aba)
    print(f"Arquivo: {arquivo} convertido com Sucesso!")

Arquivo: ../arquivos/informacao-projeto.xlsx convertido com Sucesso!
Arquivo: ../arquivos/dir1/informacao-pessoa.json convertido com Sucesso!
Arquivo: ../arquivos/dir2/informacoes-contatos.txt convertido com Sucesso!
Arquivo: ../arquivos/informacao-cargo.txt convertido com Sucesso!
Arquivo: ../arquivos/informacao-empresas.csv convertido com Sucesso!
Arquivo: ../arquivos/informacao-projeto.xlsx convertido com Sucesso!
Arquivo: ../arquivos/dir2/dir2.1/informacao-salario.csv convertido com Sucesso!
Arquivo: ../arquivos/dir2/informacao-endereco.json convertido com Sucesso!
Arquivo: ../arquivos/dir1/informacao-departamento.xlsx convertido com Sucesso!


### Consolidação final dos dados

In [78]:
!export PYTHONPATH=$(realpath $(pwd)/..)

In [79]:
from modelo.modelo import TodasColunas

In [80]:
def lista_arquivos(diretorio: str):
    try:
        lista_arquivos = []

        for root, _dir, arquivos in os.walk(diretorio):
            for arquivo in arquivos:
                lista_arquivos.append(f"{root}/{arquivo}")

        if len(lista_arquivos) == 0:
            raise TypeError()

        return lista_arquivos

    except Exception as _:
        return False

In [81]:
def consolidacao(
    arquivos_json: list
):
    try:
        consolicacao_tmp = []

        def _carrega_json(arquivo_json: str):
            with open(arquivo_json, "r+", encoding="utf-8") as arquivo:
                return json.load(arquivo)

        for arquivo_json in arquivos_json:
            dados_json = _carrega_json(arquivo_json)

            for registro in dados_json:
                dados_consolidados = TodasColunas(**registro).dict()
                consolicacao_tmp.append(dados_consolidados)

        df_consolicacao_tmp = pd.DataFrame(consolicacao_tmp)
        caminho = "../resultados/consolidacao"
        datahora = datetime.now().strftime("%Y%m%d%H%M%S")
        nome_arquivo = f"{caminho}/consolidacao_arquivos_{datahora}"
        
        if not os.path.exists(caminho):
            Path(caminho).mkdir(parents=True, exist_ok=True)

        df_consolicacao_tmp.to_csv(
            f"{nome_arquivo}.csv",
            sep=";",
            index=False
        )
    except Exception as erro:
        print(f"Erro ao consolidar os arquivos JSONs: {erro}")

In [82]:
arquivos_json = lista_arquivos("../resultados/tmp")

In [83]:
arquivos_json

['../resultados/tmp/jsons/informacao-salario-1eb65342bf1712e6a8955cc54d194f06.json',
 '../resultados/tmp/jsons/informacao-projeto-9e66d297196a0d3890f5b13fbf4fcfb0.json',
 '../resultados/tmp/jsons/informacao-cargo-3caf2a9943b3184a0827d21653c0c92a.json',
 '../resultados/tmp/jsons/informacao-endereco-fb632e2e6af12a8cf83667cf393c8b65.json',
 '../resultados/tmp/jsons/informacao-projeto-29dbbd7314c211196e640babece75163.json',
 '../resultados/tmp/jsons/informacoes-contatos-3557ea44b75903f371ced8adac5c4699.json',
 '../resultados/tmp/jsons/informacao-pessoa-b8b87ce7a31b3bcf6629e60da81fc094.json',
 '../resultados/tmp/jsons/informacao-departamento-be1d66e3562b12db2bd25e3106fe376b.json',
 '../resultados/tmp/jsons/informacao-empresas-5386f5947c77f32f59676ae53d9c5829.json']

In [84]:
consolidacao(arquivos_json)

# Referências

+ https://docs.pydantic.dev/latest/integrations/datamodel_code_generator/